<a href="https://colab.research.google.com/github/mengcius/pytorch-learn/blob/master/8_%E9%AB%98%E9%98%B6%E6%93%8D%E4%BD%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 8.高阶操作



### where 找到满足条件的
torch.where(condition,input,other) 满足condition条件返回input，不满足返回other。

因为直接赋值c[0]=a[0]是不规则的，for循环赋值使c[i,j]=a[i,j]是在CPU上没有加速的，而要用的where()是高度GPU并行实现的，一种逻辑时序控制。

In [0]:
import torch

In [4]:
x=torch.randn(3,2)
y=torch.zeros(3,2)
x

tensor([[-1.2915, -0.8518],
        [ 0.7951,  1.6197],
        [-1.6230, -0.5936]])

In [5]:
torch.where(x>0.5,x,y) #大于0.5的元素取x中对应的元素

tensor([[0.0000, 0.0000],
        [0.7951, 1.6197],
        [0.0000, 0.0000]])

### gather 收集，查表
torch.gather(input, dim, index, out=None, sparse_grad=False)  这是通过GPU完成的。

新需求可以先看能否用Torch中的where/gather来实现，效率高点，如果不行再用python中实时控制慢慢的复制/选择等。


In [18]:
t=torch.tensor([[6,7],[8,9]])
t

tensor([[6, 7],
        [8, 9]])

In [20]:
torch.gather(t,1,torch.tensor([[0,0,1],[1,0,1]])) #被查的表，dim=1，用索引查原表的值

tensor([[6, 6, 7],
        [9, 8, 9]])

神经网络输出标签label对应了索引，如果将序号绑定到另一种序号，要查找到索引对应的label

In [59]:
prob=torch.randn(4,10) #预测4张图片的minist类别结果
idx=prob.topk(dim=1,k=3) #概率最大的前3种类别的索引
idx

torch.return_types.topk(values=tensor([[1.9269, 1.6217, 1.4724],
        [1.3163, 0.6043, 0.4540],
        [1.6907, 1.4655, 1.2851],
        [1.5658, 0.4750, 0.3527]]), indices=tensor([[4, 3, 1],
        [3, 7, 0],
        [6, 4, 9],
        [4, 8, 3]]))

In [65]:
label=torch.arange(10)+100 #假设label序号重新设置了
label.expand(4,10)

tensor([[100, 101, 102, 103, 104, 105, 106, 107, 108, 109],
        [100, 101, 102, 103, 104, 105, 106, 107, 108, 109],
        [100, 101, 102, 103, 104, 105, 106, 107, 108, 109],
        [100, 101, 102, 103, 104, 105, 106, 107, 108, 109]])

In [66]:
torch.gather(label.expand(4,10),1,idx[1]) #重新查找到索引对应的label

tensor([[104, 103, 101],
        [103, 107, 100],
        [106, 104, 109],
        [104, 108, 103]])